In [5]:
import pandas as pd
import os


DATA_DIR = '../data/seattle'

In [6]:
df_calendar = pd.read_parquet(os.path.join(DATA_DIR, 'calendar.parquet'))
df_listings = pd.read_parquet(os.path.join(DATA_DIR, 'listings.parquet'))
df_reviews = pd.read_parquet(os.path.join(DATA_DIR, 'reviews.parquet'))

display('df_calendar',df_calendar.head(2))
display('df_listings',df_listings.head(2)) 
display('df_reviews',df_reviews.head(2))

'df_calendar'

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,9419,2024-12-23,f,$62.00,NaN,2,90
1,9419,2024-12-24,f,$62.00,NaN,2,90


'df_listings'

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6606,https://www.airbnb.com/rooms/6606,20241223022600,2024-12-23,city scrape,"Fab, private seattle urban cottage!","This tiny cottage is only 15x10, but it has ev...","A peaceful yet highly accessible neighborhood,...",https://a0.muscache.com/pictures/45742/21116d7...,14942,...,4.77,4.88,4.57,str-opli-19-002622,f,2,2,0,0,0.86
1,9419,https://www.airbnb.com/rooms/9419,20241223022600,2024-12-23,city scrape,Glorious sun room w/ memory foambed,This beautiful double room features sun filled...,"Lots of restaurants (see our guide book) bars,...",https://a0.muscache.com/pictures/56645186/e5fb...,30559,...,4.89,4.68,4.67,Exempt,f,11,0,11,0,1.19


'df_reviews'

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,9419,71022,2010-07-30,166478,Beth,"If you love art, animals, and yoga, this is th..."
1,9419,83099,2010-08-23,165949,Erica,I only overnighted here but I loved the offbea...


## Initial Ranker Example


In [11]:
import importlib
importlib.reload(ListingRanker)

ImportError: module ListingRanker not in sys.modules

In [15]:
import sys
sys.path.append('../src')
import listing_ranker
import importlib
importlib.reload(listing_ranker)
from listing_ranker import ListingRanker

# Example user ID from the Seattle dataset
example_user_id = 166478

# Get this user's review history
user_history = df_reviews[df_reviews['reviewer_id'] == example_user_id]

print(f"Found {len(user_history)} reviews for user {example_user_id}")

# Initialize the ranker
ranker = ListingRanker(model='phi3')

# Get ranked recommendations
ranked_recommendations = ranker.rank_listings(
    candidate_listings=df_listings,
    user_history=user_history
)

print("\nTop 5 recommended listings:")
for idx, (_, listing) in enumerate(ranked_recommendations[:5].iterrows(), 1):
    print(f"\nRank {idx}:")
    print(f"  Name: {listing['name']}")
    print(f"  Price: ${listing['price']}/night")
    print(f"  URL: {listing['listing_url']}")


Found 2 reviews for user 166478
user history:         listing_id        id        date  reviewer_id reviewer_name  \
0             9419     71022  2010-07-30       166478          Beth   
127563    13273883  89885018  2016-07-30       166478          Beth   

                                                 comments  
0       If you love art, animals, and yoga, this is th...  
127563  Great location. Easy bus commute to downtown. ...  
********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************

### Evaluator 

In [3]:
import os
import sys
notebook_dir = os.path.abspath(os.path.dirname('__file__'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(project_root)

from src.listing_ranker import ListingRanker
from src.recommender_evaluator import RecommenderEvaluator
import pandas as pd
# Set up data directory
DATA_DIR = '../data/seattle'

# Load the data
df_calendar = pd.read_parquet(os.path.join(DATA_DIR, 'calendar.parquet'))
df_listings = pd.read_parquet(os.path.join(DATA_DIR, 'listings.parquet'))
df_reviews = pd.read_parquet(os.path.join(DATA_DIR, 'reviews.parquet'))

# Initialize the ListingRanker
ranker = ListingRanker(model='phi3')

# Initialize the RecommenderEvaluator
evaluator = RecommenderEvaluator(
    ranker=ranker,
    df_listings=df_listings,
    df_reviews=df_reviews,
    metrics=['ndcg', 'precision', 'recall', 'diversity', 'coverage']
)

# Prepare holdout data
holdout_data = evaluator.prepare_holdout_set(
    test_size=0.2,  # 20% of users in test set
    min_reviews=3,  # Users must have at least 3 reviews
    random_state=42
)

# Run evaluation
results = evaluator.evaluate(
    holdout_data=holdout_data,
    k=10,  # Evaluate top 10 recommendations
    sample_size=20  # Evaluate 20 users (for faster testing)
)

# Print results
print("\nEvaluation Results:")
for metric, value in results['mean_metrics'].items():
    print(f"{metric}: {value:.4f}")

# Optionally plot results
evaluator.plot_results(pd.DataFrame([results['mean_metrics']]))

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcWriteOptions size changed, may indicate binary incompatibility. Expected 72 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow.lib.IpcReadOptions size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject


Found 12744 users with at least 3 reviews


Evaluating users:   0%|          | 0/20 [00:00<?, ?it/s]

user history:         listing_id                  id        date  reviewer_id reviewer_name  \
351258    48956729  602969059256108197  2022-04-11    447479720         Ahvie   
318450    41381981  740719905105173947  2022-10-18    447479720         Ahvie   

                                  comments  
351258  Great communication. Good privacy.  
318450                   Very professional  
************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************ Index(

Evaluating users:   5%|▌         | 1/20 [00:11<03:32, 11.16s/it]

...

Evaluating users:  95%|█████████▌| 19/20 [03:18<00:11, 11.91s/it]

Chunk 100 received
op=pairwise_llm_ranking. generate_response finished. response=
Heydea
	
In a)
The R. The textbook, Criminal Psychology|

What ifrater, which in English to their business:  

 
 Hello, ork as theta PhD

"I's "pestering]
Question> 
B-a? I amendation of a)
withdrawring. Inhenceforstayed_page10. This document infection
Ranking error: Expecting value: line 2 column 1 (char 1)
user history:         listing_id         id        date  reviewer_id reviewer_name  \
128532    11356904  128146980  2017-01-22    106179769        Alicia   
298004    36603045  526929583  2019-09-09    106179769        Alicia   

                                                 comments  
128532  The Maple Apartments are perfectly located bet...  
298004  This apartment is a gem, and Dan is a very acc...  
****************************************************************************************************************************************************************************************************

Evaluating users: 100%|██████████| 20/20 [03:32<00:00, 10.62s/it]

Chunk 100 received
op=pairwise_llm_ranking. generate_response finished. response=
You are you's 'Sedentails of a. The Greatestate, weaving myofdumpling to support?  

Given the document/money-taking_Brand newcomparative analysis, and Ivrawny0things williams, 
C\!")]
economicsian in REDI apologizing. (South POSITION: _-10national Court of a) Re
Ranking error: Expecting value: line 2 column 1 (char 1)

Evaluation Results:


KeyError: 'mean_metrics'

In [10]:
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.0 MB/s eta 0:00:00


In [14]:

# Process results - ensure we're only averaging numeric values
processed_results = {}
for metric, values in results.items():
    if isinstance(values, (list, np.ndarray)):  # If it's a list of values
        try:
            # Convert to float if possible and calculate mean
            numeric_values = [float(v) for v in values if str(v).replace('.','',1).isdigit()]
            if numeric_values:
                processed_results[metric] = np.mean(numeric_values)
            else:
                processed_results[metric] = 0.0
        except (ValueError, TypeError):
            # If conversion fails, skip this metric
            continue
    elif isinstance(values, (int, float)):  # If it's already a single numeric value
        processed_results[metric] = values
    else:
        # Store non-numeric values as is (like timestamp)
        processed_results[metric] = values

# Add metadata
processed_results['timestamp'] = "now" #datetime.now().strftime("%Y-%m-%d %H:%M:%S")
processed_results['k'] = 10
processed_results['sample_size'] = 20

# Print results
print("\nEvaluation Results:")
for metric, value in processed_results.items():
    if metric not in ['timestamp', 'k', 'sample_size']:
        if isinstance(value, (int, float)):
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value}")

# Plot results - only include numeric metrics
numeric_results = {k: v for k, v in processed_results.items() 
                  if isinstance(v, (int, float)) and k not in ['k', 'sample_size']}
#evaluator.plot_results(pd.DataFrame([numeric_results]))


Evaluation Results:
ndcg: 0.0000
precision: 0.0000
recall: 0.0000
diversity: 0.6791
coverage: 0.0617
latency: 10.6089


In [15]:
# Create multiple rankers to compare
models_dict = {
    'phi3': ListingRanker(model='phi4'),
    'random': RandomRanker(),
    'popularity': PopularityRanker(df_reviews)
}

# Compare the models
comparison_results = evaluator.compare_models(
    models_dict=models_dict,
    holdout_data=holdout_data,
    k=10,
    sample_size=20
)

# Plot the comparison
evaluator.plot_results(comparison_results)

NameError: name 'RandomRanker' is not defined